In [ ]:
import gym
import time
import random
import numpy as np

In [14]:
class Qlearner():
    def __init__(self, env, alpha = 0.01, gamma = 0.1):        
        self.numStates = env.observation_space.n
        self.numActions = env.action_space.n
        self.q = [ [0.0]*self.numActions for i in range(self.numStates) ]
        
        self.state = 0
        self.prevState = 0
        self.alpha = alpha
        self.gamma = gamma
        self.env = env
        self.env.reset()
        self.score = 0
        
    def fromFile(self, fName):
        f = open(fName, "r")
        qstr = f.readline().strip("\n")
        self.q = eval(qstr)
        self.q = np.asarray(eval(qstr))
        f.close()
        
    def toFile(self, fName):
        f = open(fName, 'w+')
        f.write(str(self.q) + "\n")
        f.close()
    
    def exploit(self, epsilon = 0.1, kill = False):
        """"Figure out what to do based upon value function"""
        random = np.random.random()
        self.prevState = self.state

        #Epsilon greedy
        if(random < epsilon):
            #Behave randomly
            actionTaken = self.env.action_space.sample() # Picks a random action
        else:
            #Behave optimally
            actionTaken = np.argmax(self.q[self.state])
        state, reward, done, inf = self.env.step(actionTaken)
        self.state = state
                
#         Negative reward for falling in hole.
#         if done and reward == 0:
#             reward = -1
            
        reward -= 0.1
        self.score += reward
        
        #Update Q(s_t,a_t)
        Qlearned = reward + self.gamma*np.max(self.q[self.state])
        self.q[self.prevState][actionTaken] += self.alpha*( Qlearned- self.q[self.prevState][actionTaken])
        
        return reward, done
    
    
    #Only works with FrozenLake 4x4
    def visualize(self):
        dirs = ['<', 'v', '>', '^']
        for i in range(4):
            row = ""
            for j in range(4):
                c = np.argmax(self.q[i*4 + j])
                row += dirs[c]
            print(row)

In [ ]:
envName = 'FrozenLake-v0'
env = gym.make(envName)
start = time.time()
ql = Qlearner(env, gamma = 0.99)

#Loads previous matrix from file into the Q-learner
# ql.fromFile(envName)

runTime = 60*0.01
start = time.time()
itters = 0

while time.time() - start < runTime:
    done = False
    ql.env.reset()
    t = 1
    kill = False
    while not done:
        if t > 1000:
            kill = True
        rew, done = ql.exploit(1.0)
        t += 1
    
    itters += 1
print("Itterations completed: {}".format(itters))
print(ql.score/itters)

ql.score = 0.0
testItters = 10
for i in range(testItters):
    done = False
    ql.env.reset()
    t = 1
    while not done:
        t += 1
        rew, done = ql.exploit(0.0)
        if t > 1000:
            done = True
            
print(ql.score/testItters)
ql.toFile(envName)


In [ ]:
env = gym.make(envName)
ql = Qlearner(env)
ql.fromFile(envName)

done = False
t = 0

while not done:
    rew, done = ql.exploit(0.0)
    ql.env.render()
    if t > 1000:
        done = True
    
print(ql.score)


In [17]:
#Viewing the sim
envName = "FrozenLake-v0"
env = gym.make(envName)
ql = Qlearner(env)
ql.fromFile(envName)
print("Loaded")

testItters = 1
for i in range(testItters):
    ql.env.reset()
    done = False
    while not done:
        rew, done = ql.exploit(1.0)
        ql.env.render()
        
print(ql.score/ testItters)

[2016-08-11 19:14:09,561] Making new env: FrozenLake-v0


Loaded
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Right)
0.3


In [ ]:
%matplotlib inline

step = 0.1
x = [0]
y = [0]

while True:
    x.append(x[-1]+step) 
    y.append(math.sqrt(x[-1]))
    plt.plot(pd.Series(data= y, index=x))
    display.display(plt.gcf())
    plt.clf() #removes old line, maintains color
    display.clear_output(wait=True)
    time.sleep(1)

In [ ]:
env = gym.make(envName)
ql = Qlearner(env)
ql.fromFile(envName)
ql.visualize()
print("\nOLD")
print("""v>v<
vvvv
>vvv
v>>v""")